# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,1.79,78,97,1.34,US,1675825380
1,1,bredasdorp,-34.5322,20.0403,18.25,81,7,2.93,ZA,1675825592
2,2,batemans bay,-35.7167,150.1833,20.61,84,100,0.94,AU,1675825593
3,3,lebu,-37.6167,-73.6500,14.88,88,0,7.79,CL,1675825043
4,4,bay roberts,47.5999,-53.2648,-2.70,85,100,7.15,CA,1675825594


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.50,
    color = "City",
    size = "Humidity",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,1.79,78,97,1.34,US,1675825380
1,1,bredasdorp,-34.5322,20.0403,18.25,81,7,2.93,ZA,1675825592
2,2,batemans bay,-35.7167,150.1833,20.61,84,100,0.94,AU,1675825593
3,3,lebu,-37.6167,-73.6500,14.88,88,0,7.79,CL,1675825043
4,4,bay roberts,47.5999,-53.2648,-2.70,85,100,7.15,CA,1675825594
...,...,...,...,...,...,...,...,...,...,...
536,536,at-bashi,41.1702,75.8106,-15.87,61,37,1.94,KG,1675825899
537,537,marienburg,54.0359,19.0266,-0.62,87,95,2.89,PL,1675825899
538,538,laguna,38.4210,-121.4238,11.62,76,0,0.00,US,1675825900
539,539,sikasso,10.9167,-7.0000,25.17,20,43,1.37,ML,1675825900


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,albany,US,42.6001,-73.9662,78,
1,bredasdorp,ZA,-34.5322,20.0403,81,
2,batemans bay,AU,-35.7167,150.1833,84,
3,lebu,CL,-37.6167,-73.6500,88,
4,bay roberts,CA,47.5999,-53.2648,85,
...,...,...,...,...,...,...
536,at-bashi,KG,41.1702,75.8106,61,
537,marienburg,PL,54.0359,19.0266,87,
538,laguna,US,38.4210,-121.4238,76,
539,sikasso,ML,10.9167,-7.0000,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError) as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
albany - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
batemans bay - nearest hotel: Esplanade Hotel
lebu - nearest hotel: No hotel found
bay roberts - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
coquimbo - nearest hotel: Hotel Iberia
chitrakonda - nearest hotel: Bala Lodge
turukhansk - nearest hotel: Гостиница ОАО "Туруханскэнерго"
longyearbyen - nearest hotel: The Vault
marbella - nearest hotel: La morada más hermosa
mwingi - nearest hotel: Greenvast Hotel
weihe - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
esil - nearest hotel: Рахат
la ronge - nearest hotel: La Ronge Hotel and Suites
ponta delgada - nearest hotel: Hotel Talisman
harbour breton - nearest hotel: No hotel found
lelydorp - nearest hotel: New Ever The Lux Hotel
lorengau - nearest hotel: Seeadler Bay Hotel
olinda - nearest hotel: Fidji Motel
port elizabeth - nearest hotel: Waterford Hotel
leh - nearest hotel: Kang Lha

,City,Country,Lat,Lng,Humidity,Hotel Name
0,albany,US,42.6001,-73.9662,78,No hotel found
1,bredasdorp,ZA,-34.5322,20.0403,81,Victoria Hotel
2,batemans bay,AU,-35.7167,150.1833,84,Esplanade Hotel
3,lebu,CL,-37.6167,-73.6500,88,No hotel found
4,bay roberts,CA,47.5999,-53.2648,85,No hotel found
...,...,...,...,...,...,...
536,at-bashi,KG,41.1702,75.8106,61,Tash Rabat
537,marienburg,PL,54.0359,19.0266,87,Stary Malbork
538,laguna,US,38.4210,-121.4238,76,Holiday Inn Express & Suites
539,sikasso,ML,10.9167,-7.0000,20,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.50,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)